# SciPy Integration and ODEs

## Learning Objectives

By the end of this notebook, you will be able to:

1. Use `quad` for numerical integration of 1D functions
2. Perform multidimensional integration with `dblquad` and `tplquad`
3. Solve initial value problems with `solve_ivp`
4. Handle systems of ODEs and stiff problems
5. Apply integration and ODE solving to scientific problems

---

## 1. Introduction to Numerical Integration

Numerical integration (quadrature) approximates definite integrals when analytical solutions are difficult or impossible to find. SciPy provides robust integration routines in `scipy.integrate`.

In [ ]:
import numpy as np
from scipy import integrate
from scipy.integrate import quad, dblquad, tplquad, solve_ivp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Set random seed for reproducibility
np.random.seed(42)

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

print("SciPy integration and ODE tools loaded!")

---

## 2. Single Integrals with quad

The `quad` function performs adaptive quadrature for single integrals.

### 2.1 Basic Integration

In [ ]:
# Example 1: Integrate x² from 0 to 2
# Analytical result: ∫x² dx from 0 to 2 = [x³/3]₀² = 8/3 ≈ 2.667

def f1(x):
    return x**2

result, error = quad(f1, 0, 2)

print("Basic Integration Examples")
print("=" * 45)
print(f"\n∫₀² x² dx")
print(f"  Numerical result: {result:.10f}")
print(f"  Analytical result: {8/3:.10f}")
print(f"  Absolute error estimate: {error:.2e}")

# Example 2: Integrate sin(x) from 0 to π
# Analytical result: [-cos(x)]₀π = -(-1) - (-1) = 2

result2, error2 = quad(np.sin, 0, np.pi)

print(f"\n∫₀π sin(x) dx")
print(f"  Numerical result: {result2:.10f}")
print(f"  Analytical result: 2.0")
print(f"  Absolute error estimate: {error2:.2e}")

In [ ]:
# Visualize the integration
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: x²
x = np.linspace(0, 2, 100)
ax1 = axes[0]
ax1.plot(x, x**2, 'b-', linewidth=2, label='f(x) = x²')
ax1.fill_between(x, 0, x**2, alpha=0.3, label=f'Area = {8/3:.4f}')
ax1.set_xlabel('x')
ax1.set_ylabel('f(x)')
ax1.set_title('∫₀² x² dx = 8/3')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: sin(x)
x = np.linspace(0, np.pi, 100)
ax2 = axes[1]
ax2.plot(x, np.sin(x), 'b-', linewidth=2, label='f(x) = sin(x)')
ax2.fill_between(x, 0, np.sin(x), alpha=0.3, label='Area = 2.0')
ax2.set_xlabel('x')
ax2.set_ylabel('f(x)')
ax2.set_title('∫₀π sin(x) dx = 2')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.2 Integration with Parameters

In [ ]:
# Gaussian function with parameters
def gaussian(x, mu, sigma):
    """Gaussian probability density function."""
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma)**2)

# Parameters
mu = 0
sigma = 1

# Integrate from -∞ to +∞ (should equal 1 for normalized PDF)
result_full, _ = quad(gaussian, -np.inf, np.inf, args=(mu, sigma))

# Integrate within 1, 2, and 3 standard deviations
result_1sigma, _ = quad(gaussian, mu - sigma, mu + sigma, args=(mu, sigma))
result_2sigma, _ = quad(gaussian, mu - 2*sigma, mu + 2*sigma, args=(mu, sigma))
result_3sigma, _ = quad(gaussian, mu - 3*sigma, mu + 3*sigma, args=(mu, sigma))

print("Gaussian Integration (68-95-99.7 Rule)")
print("=" * 45)
print(f"∫_{-∞}^{+∞} N(0,1) dx = {result_full:.6f} (should be 1.0)")
print(f"\nProbability within:")
print(f"  ±1σ: {result_1sigma*100:.2f}% (theoretical: 68.27%)")
print(f"  ±2σ: {result_2sigma*100:.2f}% (theoretical: 95.45%)")
print(f"  ±3σ: {result_3sigma*100:.2f}% (theoretical: 99.73%)")

### 2.3 Improper Integrals

In [ ]:
# Integrals with infinite limits

# Example 1: ∫₀^∞ e^(-x) dx = 1
result1, error1 = quad(lambda x: np.exp(-x), 0, np.inf)

# Example 2: ∫_{-∞}^{+∞} e^(-x²) dx = √π
result2, error2 = quad(lambda x: np.exp(-x**2), -np.inf, np.inf)

# Example 3: ∫₀^∞ x² e^(-x) dx = 2 (Gamma function: Γ(3) = 2!)
result3, error3 = quad(lambda x: x**2 * np.exp(-x), 0, np.inf)

print("Improper Integrals")
print("=" * 50)
print(f"∫₀^∞ e^(-x) dx = {result1:.10f} (analytical: 1.0)")
print(f"∫_{-∞}^{+∞} e^(-x²) dx = {result2:.10f} (analytical: √π = {np.sqrt(np.pi):.10f})")
print(f"∫₀^∞ x² e^(-x) dx = {result3:.10f} (analytical: 2.0)")

### 2.4 Handling Singularities

In [ ]:
# Integrating functions with singularities

# Example: ∫₀¹ 1/√x dx = 2
def f_singular(x):
    return 1 / np.sqrt(x)

# quad handles integrable singularities at endpoints
result, error = quad(f_singular, 0, 1)

print("Integrating Functions with Singularities")
print("=" * 45)
print(f"∫₀¹ 1/√x dx = {result:.10f} (analytical: 2.0)")
print(f"Error estimate: {error:.2e}")

# Visualize
x = np.linspace(0.001, 1, 200)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(x, 1/np.sqrt(x), 'b-', linewidth=2, label='f(x) = 1/√x')
ax.fill_between(x, 0, 1/np.sqrt(x), alpha=0.3, label=f'Area = {result:.4f}')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_ylim(0, 20)
ax.set_title('Integration with Singularity at x=0')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

### 2.5 Integration with Points of Discontinuity

In [ ]:
# Function with a known discontinuity
def f_discontinuous(x):
    """Function with jump at x=1."""
    if x < 1:
        return x
    else:
        return x + 2

# Vectorize for plotting
f_vec = np.vectorize(f_discontinuous)

# Tell quad about the discontinuity point
result, error = quad(f_discontinuous, 0, 3, points=[1])

# Analytical result: ∫₀¹ x dx + ∫₁³ (x+2) dx = 0.5 + [x²/2 + 2x]₁³ = 0.5 + (4.5+6) - (0.5+2) = 8.5

print("Integration with Discontinuity")
print("=" * 45)
print(f"∫₀³ f(x) dx = {result:.10f}")
print(f"Analytical result: 8.5")

# Visualize
x1 = np.linspace(0, 0.999, 50)
x2 = np.linspace(1.001, 3, 100)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(x1, f_vec(x1), 'b-', linewidth=2, label='f(x)')
ax.plot(x2, f_vec(x2), 'b-', linewidth=2)
ax.plot([1], [1], 'bo', markersize=8)  # closed circle at left limit
ax.plot([1], [3], 'bo', markersize=8, fillstyle='none')  # open circle at right limit
ax.fill_between(x1, 0, f_vec(x1), alpha=0.3)
ax.fill_between(x2, 0, f_vec(x2), alpha=0.3)
ax.axvline(1, color='red', linestyle='--', alpha=0.5, label='Discontinuity at x=1')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title(f'Integration with Discontinuity (Area = {result:.2f})')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

---

## 3. Double and Triple Integrals

For multidimensional integration, use `dblquad` (2D) and `tplquad` (3D).

### 3.1 Double Integrals with dblquad

In [ ]:
# Double integral: ∫∫ f(x,y) dy dx
# Note: dblquad integrates y first (inner), then x (outer)

# Example 1: ∫₀¹ ∫₀¹ x*y dy dx = [x*y²/2]₀¹ dx = ∫₀¹ x/2 dx = 1/4
def f_xy(y, x):  # Note: y comes first!
    return x * y

result, error = dblquad(f_xy, 0, 1, 0, 1)  # x: 0 to 1, y: 0 to 1

print("Double Integration")
print("=" * 45)
print(f"∫₀¹ ∫₀¹ xy dy dx = {result:.10f} (analytical: 0.25)")
print(f"Error estimate: {error:.2e}")

# Example 2: Area of a circle using double integral
# ∫∫_D 1 dA where D is the unit circle
# In Cartesian: ∫_{-1}^{1} ∫_{-√(1-x²)}^{√(1-x²)} 1 dy dx = π

def lower_y(x):
    return -np.sqrt(1 - x**2)

def upper_y(x):
    return np.sqrt(1 - x**2)

result_circle, error_circle = dblquad(lambda y, x: 1, -1, 1, lower_y, upper_y)

print(f"\nArea of unit circle: {result_circle:.10f} (analytical: π = {np.pi:.10f})")

In [ ]:
# Visualize double integration regions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Unit square
ax1 = axes[0]
x = np.linspace(0, 1, 50)
y = np.linspace(0, 1, 50)
X, Y = np.meshgrid(x, y)
Z = X * Y

contour = ax1.contourf(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(contour, ax=ax1, label='f(x,y) = xy')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('∫₀¹ ∫₀¹ xy dy dx = 1/4')
ax1.set_aspect('equal')

# Plot 2: Unit circle
ax2 = axes[1]
theta = np.linspace(0, 2*np.pi, 100)
ax2.fill(np.cos(theta), np.sin(theta), alpha=0.5, label=f'Area = π')
ax2.plot(np.cos(theta), np.sin(theta), 'b-', linewidth=2)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('Area of Unit Circle = π')
ax2.set_aspect('equal')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.2 Triple Integrals with tplquad

In [ ]:
# Triple integral: volume of a sphere
# V = ∫∫∫ 1 dV for the unit sphere
# Analytical: V = (4/3)πr³ = (4/3)π for r=1

# Note: tplquad integrates z first, then y, then x
# f(z, y, x), x: [a, b], y: [gfun, hfun](x), z: [qfun, rfun](x, y)

def sphere_z_lower(x, y):
    r2 = x**2 + y**2
    if r2 >= 1:
        return 0
    return -np.sqrt(1 - r2)

def sphere_z_upper(x, y):
    r2 = x**2 + y**2
    if r2 >= 1:
        return 0
    return np.sqrt(1 - r2)

def sphere_y_lower(x):
    return -np.sqrt(max(0, 1 - x**2))

def sphere_y_upper(x):
    return np.sqrt(max(0, 1 - x**2))

# This is computationally intensive, so we'll use coarser tolerance
result_sphere, error_sphere = tplquad(
    lambda z, y, x: 1,  # integrand
    -1, 1,              # x limits
    sphere_y_lower, sphere_y_upper,  # y limits
    sphere_z_lower, sphere_z_upper,  # z limits
)

analytical_sphere = (4/3) * np.pi

print("Triple Integration: Volume of Unit Sphere")
print("=" * 50)
print(f"Numerical result: {result_sphere:.6f}")
print(f"Analytical result: (4/3)π = {analytical_sphere:.6f}")
print(f"Relative error: {abs(result_sphere - analytical_sphere) / analytical_sphere * 100:.4f}%")

### 3.3 Using nquad for General N-dimensional Integrals

In [ ]:
# nquad provides more flexibility for multidimensional integrals
from scipy.integrate import nquad

# Example: ∫∫ x²y dA over a triangular region
# Region: 0 <= x <= 1, 0 <= y <= x

def integrand(y, x):
    return x**2 * y

# Define limits as functions
def y_lower(x):
    return 0

def y_upper(x):
    return x

# nquad syntax: integrand, [x_limits, y_limits]
# y_limits can be functions of x
result, error = nquad(integrand, [[0, 1], [y_lower, y_upper]])

# Analytical: ∫₀¹ ∫₀ˣ x²y dy dx = ∫₀¹ x²[y²/2]₀ˣ dx = ∫₀¹ x⁴/2 dx = [x⁵/10]₀¹ = 1/10

print("nquad for Variable Limits")
print("=" * 45)
print(f"∫₀¹ ∫₀ˣ x²y dy dx = {result:.10f}")
print(f"Analytical result: 1/10 = 0.1")

---

## 4. Solving Ordinary Differential Equations (ODEs)

The `solve_ivp` function solves initial value problems for ODEs.

### 4.1 First-Order ODEs

In [ ]:
# Example 1: Exponential decay
# dy/dt = -k*y, y(0) = y0
# Analytical solution: y(t) = y0 * exp(-k*t)

def decay(t, y, k):
    """Exponential decay ODE."""
    return -k * y

# Parameters
k = 0.5
y0 = [10]  # Initial condition (must be array-like)
t_span = (0, 10)  # Time interval

# Solve ODE
solution = solve_ivp(decay, t_span, y0, args=(k,), dense_output=True)

print("Exponential Decay ODE: dy/dt = -ky")
print("=" * 45)
print(f"Initial condition: y(0) = {y0[0]}")
print(f"Decay constant k = {k}")
print(f"Solution status: {solution.message}")
print(f"Number of time steps: {len(solution.t)}")

# Evaluate solution at specific points
t_eval = np.linspace(0, 10, 100)
y_numerical = solution.sol(t_eval)[0]
y_analytical = y0[0] * np.exp(-k * t_eval)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(t_eval, y_numerical, 'b-', linewidth=2, label='Numerical')
ax1.plot(t_eval, y_analytical, 'r--', linewidth=2, label='Analytical')
ax1.plot(solution.t, solution.y[0], 'go', markersize=6, label='Solver steps')
ax1.set_xlabel('Time')
ax1.set_ylabel('y(t)')
ax1.set_title('Exponential Decay Solution')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.semilogy(t_eval, np.abs(y_numerical - y_analytical), 'b-', linewidth=2)
ax2.set_xlabel('Time')
ax2.set_ylabel('|Error|')
ax2.set_title('Numerical Error')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 4.2 Second-Order ODEs (Simple Harmonic Oscillator)

In [ ]:
# Simple Harmonic Oscillator: d²x/dt² = -ω²x
# Convert to system of first-order ODEs:
# dx/dt = v
# dv/dt = -ω²x

def harmonic_oscillator(t, state, omega):
    """Simple harmonic oscillator.
    
    state = [x, v] where x is position and v is velocity
    """
    x, v = state
    dxdt = v
    dvdt = -omega**2 * x
    return [dxdt, dvdt]

# Parameters
omega = 2 * np.pi  # frequency (1 Hz)
x0 = 1.0  # initial position
v0 = 0.0  # initial velocity
state0 = [x0, v0]
t_span = (0, 5)  # 5 seconds

# Solve
solution = solve_ivp(harmonic_oscillator, t_span, state0, args=(omega,),
                     dense_output=True, max_step=0.01)

# Evaluate solution
t_eval = np.linspace(0, 5, 500)
sol = solution.sol(t_eval)
x_num = sol[0]
v_num = sol[1]

# Analytical solution
x_anal = x0 * np.cos(omega * t_eval) + (v0 / omega) * np.sin(omega * t_eval)
v_anal = -x0 * omega * np.sin(omega * t_eval) + v0 * np.cos(omega * t_eval)

print("Simple Harmonic Oscillator: d²x/dt² = -ω²x")
print("=" * 50)
print(f"Angular frequency ω = {omega:.4f} rad/s")
print(f"Period T = {2*np.pi/omega:.4f} s")
print(f"Initial conditions: x(0) = {x0}, v(0) = {v0}")

In [ ]:
# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Position vs time
ax1 = axes[0, 0]
ax1.plot(t_eval, x_num, 'b-', linewidth=2, label='Numerical')
ax1.plot(t_eval, x_anal, 'r--', linewidth=1, label='Analytical')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Position x(t)')
ax1.set_title('Position vs Time')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Velocity vs time
ax2 = axes[0, 1]
ax2.plot(t_eval, v_num, 'g-', linewidth=2, label='Numerical')
ax2.plot(t_eval, v_anal, 'r--', linewidth=1, label='Analytical')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Velocity v(t)')
ax2.set_title('Velocity vs Time')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Phase space
ax3 = axes[1, 0]
ax3.plot(x_num, v_num, 'b-', linewidth=2, label='Trajectory')
ax3.plot(x0, v0, 'go', markersize=10, label='Start')
ax3.set_xlabel('Position x')
ax3.set_ylabel('Velocity v')
ax3.set_title('Phase Space Portrait')
ax3.legend()
ax3.set_aspect('equal')
ax3.grid(True, alpha=0.3)

# Energy conservation
KE = 0.5 * v_num**2  # kinetic energy (m=1)
PE = 0.5 * omega**2 * x_num**2  # potential energy
total_energy = KE + PE

ax4 = axes[1, 1]
ax4.plot(t_eval, KE, 'b-', linewidth=2, label='Kinetic')
ax4.plot(t_eval, PE, 'r-', linewidth=2, label='Potential')
ax4.plot(t_eval, total_energy, 'k--', linewidth=2, label='Total')
ax4.set_xlabel('Time (s)')
ax4.set_ylabel('Energy')
ax4.set_title('Energy Conservation')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 4.3 Damped Harmonic Oscillator

In [ ]:
# Damped oscillator: d²x/dt² = -ω²x - 2γ(dx/dt)
# where γ is the damping coefficient

def damped_oscillator(t, state, omega, gamma):
    """Damped harmonic oscillator."""
    x, v = state
    dxdt = v
    dvdt = -omega**2 * x - 2 * gamma * v
    return [dxdt, dvdt]

omega = 2 * np.pi
state0 = [1.0, 0.0]
t_span = (0, 5)
t_eval = np.linspace(0, 5, 500)

# Different damping regimes
damping_cases = [
    (0.5, 'Underdamped (γ < ω)'),
    (omega, 'Critically damped (γ = ω)'),
    (2 * omega, 'Overdamped (γ > ω)'),
]

fig, ax = plt.subplots(figsize=(12, 6))

for gamma, label in damping_cases:
    solution = solve_ivp(damped_oscillator, t_span, state0, args=(omega, gamma),
                         t_eval=t_eval)
    ax.plot(solution.t, solution.y[0], linewidth=2, label=label)

# Also plot undamped for reference
solution_undamped = solve_ivp(harmonic_oscillator, t_span, state0, args=(omega,),
                              t_eval=t_eval)
ax.plot(solution_undamped.t, solution_undamped.y[0], 'k--', linewidth=1, 
        alpha=0.5, label='Undamped')

ax.axhline(0, color='gray', linestyle=':', alpha=0.5)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Position x(t)')
ax.set_title('Damped Harmonic Oscillator: Different Damping Regimes')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

### 4.4 Systems of ODEs: Lorenz System

In [ ]:
# Lorenz System - a classic chaotic system
# dx/dt = σ(y - x)
# dy/dt = x(ρ - z) - y
# dz/dt = xy - βz

def lorenz(t, state, sigma, rho, beta):
    """Lorenz system of ODEs."""
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Classic Lorenz parameters (chaotic regime)
sigma = 10
rho = 28
beta = 8/3

# Initial conditions
state0 = [1.0, 1.0, 1.0]
t_span = (0, 50)

# Solve
solution = solve_ivp(lorenz, t_span, state0, args=(sigma, rho, beta),
                     dense_output=True, max_step=0.01)

t_eval = np.linspace(0, 50, 10000)
sol = solution.sol(t_eval)

print("Lorenz System (Chaotic Attractor)")
print("=" * 45)
print(f"Parameters: σ={sigma}, ρ={rho}, β={beta:.4f}")
print(f"Initial conditions: {state0}")

In [ ]:
# Visualize Lorenz attractor
fig = plt.figure(figsize=(15, 5))

# 3D trajectory
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot(sol[0], sol[1], sol[2], 'b-', linewidth=0.5, alpha=0.7)
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('Lorenz Attractor (3D)')

# X-Y projection
ax2 = fig.add_subplot(132)
ax2.plot(sol[0], sol[1], 'b-', linewidth=0.3, alpha=0.5)
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_title('X-Y Projection')
ax2.grid(True, alpha=0.3)

# X-Z projection
ax3 = fig.add_subplot(133)
ax3.plot(sol[0], sol[2], 'r-', linewidth=0.3, alpha=0.5)
ax3.set_xlabel('X')
ax3.set_ylabel('Z')
ax3.set_title('X-Z Projection')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 4.5 Stiff ODEs

In [ ]:
# Stiff ODEs have solutions with widely different time scales
# Example: Van der Pol oscillator with large μ
# d²x/dt² - μ(1-x²)(dx/dt) + x = 0

def van_der_pol(t, state, mu):
    """Van der Pol oscillator (stiff for large mu)."""
    x, v = state
    dxdt = v
    dvdt = mu * (1 - x**2) * v - x
    return [dxdt, dvdt]

# Stiff case: large mu
mu = 1000
state0 = [2.0, 0.0]
t_span = (0, 3000)

# Compare explicit vs implicit methods
print("Stiff ODE: Van der Pol with μ = 1000")
print("=" * 50)

# RK45 (explicit, not ideal for stiff)
import time
start = time.time()
sol_rk45 = solve_ivp(van_der_pol, t_span, state0, args=(mu,), method='RK45',
                     max_step=0.1)
time_rk45 = time.time() - start
print(f"RK45: {len(sol_rk45.t)} steps, {time_rk45:.2f} seconds")

# BDF (implicit, good for stiff)
start = time.time()
sol_bdf = solve_ivp(van_der_pol, t_span, state0, args=(mu,), method='BDF')
time_bdf = time.time() - start
print(f"BDF:  {len(sol_bdf.t)} steps, {time_bdf:.2f} seconds")

# Radau (implicit, good for stiff)
start = time.time()
sol_radau = solve_ivp(van_der_pol, t_span, state0, args=(mu,), method='Radau')
time_radau = time.time() - start
print(f"Radau: {len(sol_radau.t)} steps, {time_radau:.2f} seconds")

In [ ]:
# Visualize Van der Pol solution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Time series
ax1 = axes[0]
ax1.plot(sol_bdf.t, sol_bdf.y[0], 'b-', linewidth=1)
ax1.set_xlabel('Time')
ax1.set_ylabel('x(t)')
ax1.set_title(f'Van der Pol Oscillator (μ = {mu})')
ax1.grid(True, alpha=0.3)

# Phase portrait
ax2 = axes[1]
ax2.plot(sol_bdf.y[0], sol_bdf.y[1], 'b-', linewidth=0.5)
ax2.plot(state0[0], state0[1], 'go', markersize=10, label='Start')
ax2.set_xlabel('x')
ax2.set_ylabel('v = dx/dt')
ax2.set_title('Phase Portrait (Limit Cycle)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 5. Practical Examples

### 5.1 Predator-Prey Model (Lotka-Volterra)

In [ ]:
# Lotka-Volterra predator-prey equations
# dR/dt = αR - βRF   (prey growth - predation)
# dF/dt = δRF - γF   (predator growth - death)

def lotka_volterra(t, state, alpha, beta, delta, gamma):
    """Lotka-Volterra predator-prey model.
    
    R = prey (rabbits), F = predator (foxes)
    """
    R, F = state
    dRdt = alpha * R - beta * R * F
    dFdt = delta * R * F - gamma * F
    return [dRdt, dFdt]

# Parameters
alpha = 1.1   # prey growth rate
beta = 0.4    # predation rate
delta = 0.1   # predator reproduction rate
gamma = 0.4   # predator death rate

# Initial populations
R0, F0 = 10, 5  # 10 prey, 5 predators
state0 = [R0, F0]
t_span = (0, 100)

# Solve
solution = solve_ivp(lotka_volterra, t_span, state0, 
                     args=(alpha, beta, delta, gamma),
                     dense_output=True, max_step=0.1)

t_eval = np.linspace(0, 100, 1000)
sol = solution.sol(t_eval)

print("Lotka-Volterra Predator-Prey Model")
print("=" * 45)
print(f"Prey growth rate α = {alpha}")
print(f"Predation rate β = {beta}")
print(f"Predator reproduction δ = {delta}")
print(f"Predator death rate γ = {gamma}")

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Population dynamics over time
ax1 = axes[0]
ax1.plot(t_eval, sol[0], 'b-', linewidth=2, label='Prey (Rabbits)')
ax1.plot(t_eval, sol[1], 'r-', linewidth=2, label='Predators (Foxes)')
ax1.set_xlabel('Time')
ax1.set_ylabel('Population')
ax1.set_title('Population Dynamics Over Time')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Phase portrait
ax2 = axes[1]
ax2.plot(sol[0], sol[1], 'g-', linewidth=1)
ax2.plot(R0, F0, 'ko', markersize=10, label=f'Start ({R0}, {F0})')
ax2.set_xlabel('Prey Population')
ax2.set_ylabel('Predator Population')
ax2.set_title('Phase Portrait (Predator vs Prey)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Mark equilibrium point
R_eq = gamma / delta
F_eq = alpha / beta
ax2.plot(R_eq, F_eq, 'r*', markersize=15, label=f'Equilibrium ({R_eq:.1f}, {F_eq:.1f})')
ax2.legend()

plt.tight_layout()
plt.show()

### 5.2 Calculating Probability from Distribution

In [ ]:
# Calculate probability that a normally distributed variable
# falls within a certain range

from scipy import stats

# Student test scores: mean = 75, std = 10
mu = 75
sigma = 10

def normal_pdf(x):
    return stats.norm.pdf(x, mu, sigma)

# Probability of scoring between 70 and 90
prob_70_90, _ = quad(normal_pdf, 70, 90)

# Probability of scoring above 85
prob_above_85, _ = quad(normal_pdf, 85, np.inf)

# Probability of scoring below 60
prob_below_60, _ = quad(normal_pdf, -np.inf, 60)

print("Student Test Score Probabilities")
print("=" * 45)
print(f"Distribution: N({mu}, {sigma}²)")
print(f"\nP(70 ≤ score ≤ 90): {prob_70_90*100:.2f}%")
print(f"P(score > 85): {prob_above_85*100:.2f}%")
print(f"P(score < 60): {prob_below_60*100:.2f}%")

# Verify with scipy.stats CDF
print("\nVerification using CDF:")
print(f"P(70 ≤ score ≤ 90): {(stats.norm.cdf(90, mu, sigma) - stats.norm.cdf(70, mu, sigma))*100:.2f}%")

---

## Exercises

Practice what you've learned with these exercises.

### Exercise 1: Numerical Integration

The error function (erf) is defined as:
erf(x) = (2/√π) ∫₀ˣ e^(-t²) dt

1. Implement this integral using quad
2. Calculate erf(1), erf(2), and erf(3)
3. Compare with scipy.special.erf
4. Plot your implementation vs scipy's

In [ ]:
from scipy.special import erf as scipy_erf

# Your code here


<details>
<summary>Click to see solution</summary>

```python
# 1. Implement erf using quad
def my_erf(x):
    """Error function using numerical integration."""
    integrand = lambda t: np.exp(-t**2)
    result, _ = quad(integrand, 0, x)
    return (2 / np.sqrt(np.pi)) * result

# 2. Calculate specific values
print("Error Function Calculation")
print("=" * 45)
for x in [1, 2, 3]:
    my_val = my_erf(x)
    scipy_val = scipy_erf(x)
    print(f"erf({x}): my_erf = {my_val:.10f}, scipy = {scipy_val:.10f}")

# 3 & 4. Plot comparison
x_vals = np.linspace(0, 3, 50)
my_erf_vals = [my_erf(x) for x in x_vals]
scipy_erf_vals = scipy_erf(x_vals)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Function comparison
ax1 = axes[0]
ax1.plot(x_vals, my_erf_vals, 'b-', linewidth=2, label='my_erf (quad)')
ax1.plot(x_vals, scipy_erf_vals, 'r--', linewidth=2, label='scipy.special.erf')
ax1.set_xlabel('x')
ax1.set_ylabel('erf(x)')
ax1.set_title('Error Function Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Error
ax2 = axes[1]
ax2.semilogy(x_vals, np.abs(np.array(my_erf_vals) - scipy_erf_vals), 'b-', linewidth=2)
ax2.set_xlabel('x')
ax2.set_ylabel('|Difference|')
ax2.set_title('Difference Between Implementations')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
```
</details>

### Exercise 2: Double Integration

Calculate the volume under the surface z = sin(x)cos(y) over the region 0 ≤ x ≤ π, 0 ≤ y ≤ π/2.

1. Use dblquad to compute the integral
2. Verify analytically: ∫∫ sin(x)cos(y) dy dx = [-cos(x)]₀^π × [sin(y)]₀^(π/2) = 2 × 1 = 2
3. Create a 3D visualization of the surface

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
# 1. Compute using dblquad
def surface(y, x):
    return np.sin(x) * np.cos(y)

result, error = dblquad(surface, 0, np.pi, 0, np.pi/2)

print("Double Integration: Volume Under Surface")
print("=" * 50)
print(f"∫₀^π ∫₀^(π/2) sin(x)cos(y) dy dx")
print(f"Numerical result: {result:.10f}")
print(f"Analytical result: 2.0")
print(f"Error estimate: {error:.2e}")

# 3. 3D visualization
x = np.linspace(0, np.pi, 50)
y = np.linspace(0, np.pi/2, 50)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y)

fig = plt.figure(figsize=(14, 5))

# 3D surface plot
ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z = sin(x)cos(y)')
ax1.set_title(f'Surface (Volume = {result:.4f})')
fig.colorbar(surf, ax=ax1, shrink=0.5)

# Contour plot
ax2 = fig.add_subplot(122)
contour = ax2.contourf(X, Y, Z, levels=20, cmap='viridis')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('Contour Plot of z = sin(x)cos(y)')
plt.colorbar(contour, ax=ax2)

plt.tight_layout()
plt.show()
```
</details>

### Exercise 3: Projectile Motion with Air Resistance

Solve the projectile motion equations with quadratic air resistance:
- d²x/dt² = -k*v*vx where v = √(vx² + vy²)
- d²y/dt² = -g - k*v*vy

Initial conditions: x(0)=0, y(0)=0, vx(0)=v0*cos(θ), vy(0)=v0*sin(θ)

1. Implement the ODE system
2. Solve for v0=50 m/s, θ=45°, k=0.01
3. Compare with no air resistance
4. Plot both trajectories

In [ ]:
g = 9.81  # m/s²

# Your code here


<details>
<summary>Click to see solution</summary>

```python
# 1. Implement ODE system
def projectile_drag(t, state, k, g):
    """Projectile with quadratic air resistance.
    
    state = [x, y, vx, vy]
    """
    x, y, vx, vy = state
    v = np.sqrt(vx**2 + vy**2)
    
    dxdt = vx
    dydt = vy
    dvxdt = -k * v * vx
    dvydt = -g - k * v * vy
    
    return [dxdt, dydt, dvxdt, dvydt]

def projectile_no_drag(t, state, g):
    """Projectile without air resistance."""
    x, y, vx, vy = state
    return [vx, vy, 0, -g]

# Hit ground event
def hit_ground(t, state, *args):
    return state[1]  # y = 0
hit_ground.terminal = True
hit_ground.direction = -1

# 2. Parameters
v0 = 50  # m/s
theta = np.radians(45)
k = 0.01  # drag coefficient

vx0 = v0 * np.cos(theta)
vy0 = v0 * np.sin(theta)
state0 = [0, 0, vx0, vy0]
t_span = (0, 20)

# Solve with drag
sol_drag = solve_ivp(projectile_drag, t_span, state0, args=(k, g),
                     events=hit_ground, dense_output=True, max_step=0.01)

# Solve without drag
sol_no_drag = solve_ivp(projectile_no_drag, t_span, state0, args=(g,),
                        events=hit_ground, dense_output=True, max_step=0.01)

print("Projectile Motion Analysis")
print("=" * 45)
print(f"Initial velocity: {v0} m/s at {np.degrees(theta):.0f}°")
print(f"Drag coefficient: k = {k}")
print(f"\nResults:")
print(f"  With drag:    Range = {sol_drag.y[0, -1]:.2f} m, Time = {sol_drag.t[-1]:.2f} s")
print(f"  Without drag: Range = {sol_no_drag.y[0, -1]:.2f} m, Time = {sol_no_drag.t[-1]:.2f} s")

# 4. Plot trajectories
fig, ax = plt.subplots(figsize=(12, 6))

t_drag = np.linspace(0, sol_drag.t[-1], 500)
t_no_drag = np.linspace(0, sol_no_drag.t[-1], 500)

traj_drag = sol_drag.sol(t_drag)
traj_no_drag = sol_no_drag.sol(t_no_drag)

ax.plot(traj_drag[0], traj_drag[1], 'b-', linewidth=2, label='With air resistance')
ax.plot(traj_no_drag[0], traj_no_drag[1], 'r--', linewidth=2, label='No air resistance')
ax.set_xlabel('Horizontal Distance (m)')
ax.set_ylabel('Height (m)')
ax.set_title('Projectile Motion: Effect of Air Resistance')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, None)
plt.show()
```
</details>

### Exercise 4: SIR Epidemic Model

The SIR model describes the spread of disease:
- dS/dt = -β*S*I/N (susceptible -> infected)
- dI/dt = β*S*I/N - γ*I (infected: new cases - recoveries)
- dR/dt = γ*I (recovered)

Where N = S + I + R is total population.

1. Implement the SIR model
2. Solve for N=1000, initial I=1, β=0.3, γ=0.1
3. Calculate R0 = β/γ (basic reproduction number)
4. Plot S, I, R over time

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
# 1. Implement SIR model
def sir_model(t, state, beta, gamma, N):
    """SIR epidemic model."""
    S, I, R = state
    
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    
    return [dSdt, dIdt, dRdt]

# 2. Parameters
N = 1000
beta = 0.3  # transmission rate
gamma = 0.1  # recovery rate

S0 = N - 1
I0 = 1
R0_init = 0
state0 = [S0, I0, R0_init]
t_span = (0, 200)

# Solve
solution = solve_ivp(sir_model, t_span, state0, args=(beta, gamma, N),
                     dense_output=True, max_step=0.5)

t_eval = np.linspace(0, 200, 1000)
sol = solution.sol(t_eval)

# 3. Calculate R0
R0 = beta / gamma

print("SIR Epidemic Model")
print("=" * 45)
print(f"Population N = {N}")
print(f"Transmission rate β = {beta}")
print(f"Recovery rate γ = {gamma}")
print(f"\nBasic reproduction number R₀ = β/γ = {R0:.2f}")
print(f"(Epidemic spreads if R₀ > 1)")

# Peak infections
peak_idx = np.argmax(sol[1])
print(f"\nPeak infected: {sol[1][peak_idx]:.0f} at day {t_eval[peak_idx]:.0f}")
print(f"Final recovered: {sol[2][-1]:.0f} ({sol[2][-1]/N*100:.1f}% of population)")

# 4. Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Population over time
ax1 = axes[0]
ax1.plot(t_eval, sol[0], 'b-', linewidth=2, label='Susceptible')
ax1.plot(t_eval, sol[1], 'r-', linewidth=2, label='Infected')
ax1.plot(t_eval, sol[2], 'g-', linewidth=2, label='Recovered')
ax1.axhline(sol[1][peak_idx], color='red', linestyle='--', alpha=0.5)
ax1.set_xlabel('Time (days)')
ax1.set_ylabel('Number of People')
ax1.set_title(f'SIR Epidemic Model (R₀ = {R0:.1f})')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Phase portrait (S vs I)
ax2 = axes[1]
ax2.plot(sol[0], sol[1], 'b-', linewidth=2)
ax2.plot(S0, I0, 'go', markersize=10, label='Start')
ax2.plot(sol[0][-1], sol[1][-1], 'ro', markersize=10, label='End')
ax2.set_xlabel('Susceptible')
ax2.set_ylabel('Infected')
ax2.set_title('Phase Portrait')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
```
</details>

### Exercise 5: RC Circuit Discharge

An RC circuit follows: dV/dt = -V/(RC)

With initial voltage V(0) = V0, the solution is V(t) = V0 * exp(-t/RC).

1. Solve numerically for R = 1000 Ω, C = 10 μF, V0 = 5 V
2. Calculate the time constant τ = RC
3. Find the time when V drops to 1/e of V0 (should equal τ)
4. Compare numerical and analytical solutions

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
# 1. Define and solve ODE
def rc_discharge(t, V, R, C):
    return -V / (R * C)

# Parameters
R = 1000      # Ohms
C = 10e-6     # Farads (10 μF)
V0 = 5        # Volts

# 2. Time constant
tau = R * C

# Solve for several time constants
t_span = (0, 5 * tau)
state0 = [V0]

solution = solve_ivp(rc_discharge, t_span, state0, args=(R, C),
                     dense_output=True, max_step=tau/100)

t_eval = np.linspace(0, 5 * tau, 500)
V_numerical = solution.sol(t_eval)[0]
V_analytical = V0 * np.exp(-t_eval / tau)

# 3. Find time when V = V0/e
target_V = V0 / np.e
idx = np.argmin(np.abs(V_numerical - target_V))
t_at_target = t_eval[idx]

print("RC Circuit Discharge")
print("=" * 45)
print(f"Resistance R = {R} Ω")
print(f"Capacitance C = {C*1e6:.1f} μF")
print(f"Initial voltage V0 = {V0} V")
print(f"\nTime constant τ = RC = {tau*1000:.2f} ms")
print(f"Time when V = V0/e: {t_at_target*1000:.2f} ms (should = τ)")

# 4. Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(t_eval*1000, V_numerical, 'b-', linewidth=2, label='Numerical')
ax1.plot(t_eval*1000, V_analytical, 'r--', linewidth=2, label='Analytical')
ax1.axhline(V0/np.e, color='gray', linestyle=':', alpha=0.5, label=f'V0/e = {V0/np.e:.2f} V')
ax1.axvline(tau*1000, color='green', linestyle=':', alpha=0.5, label=f'τ = {tau*1000:.2f} ms')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Voltage (V)')
ax1.set_title('RC Circuit Discharge')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.semilogy(t_eval*1000, np.abs(V_numerical - V_analytical), 'b-', linewidth=2)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('|Error| (V)')
ax2.set_title('Numerical Error')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
```
</details>

---

## Summary

In this notebook, you learned:

1. **Single Integration with quad**
   - Basic integration with `quad`
   - Handling parameters, infinite limits, and singularities
   - Integration over discontinuous functions

2. **Multiple Integration**
   - `dblquad` for double integrals
   - `tplquad` for triple integrals
   - `nquad` for general N-dimensional integrals

3. **Solving ODEs with solve_ivp**
   - First-order ODEs
   - Converting higher-order ODEs to systems
   - Systems of coupled ODEs
   - Stiff ODEs and appropriate solvers

4. **Practical Applications**
   - Harmonic oscillators (simple and damped)
   - Lorenz chaotic system
   - Lotka-Volterra predator-prey model
   - Probability calculations

---

## Next Steps

Congratulations on completing the SciPy module! You now have a solid foundation in:
- Statistical analysis
- Interpolation and curve fitting
- Optimization
- Integration and differential equations

Continue exploring:
- **scipy.signal** for signal processing
- **scipy.fft** for Fourier transforms
- **scipy.sparse** for sparse matrix operations
- **scipy.linalg** for advanced linear algebra